# Data wrangling over excel files

---



In [181]:
!pip install polars 
!pip install xlsx2csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [182]:
import polars as pl
import xlrd as xls
import numpy as np

> ## **A prior implementation**: descriptions are included

In [306]:
file = "/content/data_income/existencias-animales.xlsx"
file = "/content/data_income/actividad-principal-region.xlsx"
file = "/content/data_income/existencias-colmenas.xlsx"

In [307]:
get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot]
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
del_duplicate = lambda xs: [name for (name, pibot) in zip(xs, ["><"]+xs) if name != pibot]
## Filling up empty space within the row
filling_data = lambda xs: [ xs[i-1] if ( xs[i]=="" and i>0 ) else xs[i] for i in range(len(xs)) ]

Obtaining indexes for sectoring data content

In [308]:
work_book = xls.open_workbook(file)
work_book = work_book.sheet_by_index(0)

column = 1

array_col = work_book.col_values(column)[1:]
#array_col = del_duplicate(array_col)

*a,ini,fin = get_indexes("", array_col)

## Filling up empty space within the row
array_col = filling_data(array_col)

## Keeping range that it cotains data
array_col = array_col[ini: fin]
print(array_col)

['Total Nacional - Región', 'Total Nacional', 'Región de Arica y Parinacota', 'Región de Tarapacá ', 'Región de Antofagasta', 'Región de Atacama', 'Región de Coquimbo', 'Región de Valparaíso', 'Región Metropolitana de Santiago', "Región del Libertador General Bernardo O'Higgins", 'Región del Maule', 'Región de Ñuble', 'Región del Biobío ', 'Región de La Araucanía', 'Región de Los Ríos', 'Región de Los Lagos', 'Región de Aysén del General Carlos Ibáñez del Campo', 'Región de Magallanes y de la Antártica Chilena']


Selecting columns as arrays
Sizing data of the columns
Data normalization
Storing arrays into fields

This process will finish when there are no more columns with data

In [309]:
fields = np.array(array_col)

column=2
while True:
  try:
    array_col = work_book.col_values(column)[1:]
    #array_col = del_duplicate(array_col)
    
    ## Filling up empty space within the row
    array_col = filling_data(array_col)

    array_col = array_col[ini:fin]
    column+=1

    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.replace(array_col, " ", "_")

    #print(array_col)
    fields = np.row_stack((fields, array_col))
  except:
    print("Ending column")
    break
print(fields)

Ending column
[['Total Nacional - Región' 'Total Nacional'
  'Región de Arica y Parinacota' 'Región de Tarapacá '
  'Región de Antofagasta' 'Región de Atacama' 'Región de Coquimbo'
  'Región de Valparaíso' 'Región Metropolitana de Santiago'
  "Región del Libertador General Bernardo O'Higgins" 'Región del Maule'
  'Región de Ñuble' 'Región del Biobío ' 'Región de La Araucanía'
  'Región de Los Ríos' 'Región de Los Lagos'
  'Región de Aysén del General Carlos Ibáñez del Campo'
  'Región de Magallanes y de la Antártica Chilena']
 ['número_de_upa3,4' '5909.0' '6.0' '8.0' '7.0' '30.0' '297.0' '186.0'
  '209.0' '387.0' '686.0' '444.0' '947.0' '1690.0' '456.0' '536.0' '29.0'
  '1.0']
 ['número_de_colmenas' '333037.0' '53.0' '48.0' '25.0' '359.0' '6448.0'
  '24658.0' '29572.0' '50291.0' '63887.0' '38936.0' '36048.0' '38489.0'
  '16781.0' '27070.0' '366.0' '6.0']
 ['número_de_upa4' '4997.0' '5.0' '4.0' '5.0' '24.0' '230.0' '165.0'
  '188.0' '353.0' '597.0' '386.0' '824.0' '1402.0' '398.0' '402.

Removing rows with lots of null values to get title for the dataframe

In [316]:
array_to_dataframe = np.transpose(fields)

pos = 0
normal_long = len(array_to_dataframe[pos])

while True:
  titles = array_to_dataframe[pos]
  out_arr = np.char.count(titles, sub ='', start=1, end=1)
  actual_long = np.sum(out_arr)
  if not len(out_arr) == actual_long :
    array_to_dataframe = array_to_dataframe[1:]
  else:
    break

titles = np.char.lower(titles)
titles = np.char.replace(titles, " ", "_")

for i in range(len(titles)):
  titles[i] = titles[i] +"_"+ str(i)

print(titles)

['total_nacional_-_región' 'número_de_upa3,4_1' 'número_de_colmenas_2'
 'número_de_upa4_3' 'número_de_colmenas_4' 'número_de_upa4_5'
 'número_de_colmenas_6']


Showing the datafram

In [317]:
df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
pl.from_numpy
df_polar = df_polar[1:]
print(df_polar)

file = file.replace("_income","_outcome")
df_polar.write_csv(file.split('.')[0]+'.csv')


shape: (17, 7)
┌────────────┬────────────┬────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ total_naci ┆ número_de_ ┆ número_de_ ┆ número_de_up ┆ número_de_co ┆ número_de_up ┆ número_de_co │
│ onal_-_reg ┆ upa3,4_1   ┆ colmenas_2 ┆ a4_3         ┆ lmenas_4     ┆ a4_5         ┆ lmenas_6     │
│ ión        ┆ ---        ┆ ---        ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│ ---        ┆ str        ┆ str        ┆ str          ┆ str          ┆ str          ┆ str          │
│ str        ┆            ┆            ┆              ┆              ┆              ┆              │
╞════════════╪════════════╪════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ Total      ┆ 5909.0     ┆ 333037.0   ┆ 4997.0       ┆ 306758.0     ┆ 995.0        ┆ 26279.0      │
│ Nacional   ┆            ┆            ┆              ┆              ┆              ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌

> ## **Object-oriented programming**

In [188]:
!pip install polars 
!pip install xlsx2csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [189]:
import polars as pl
import xlrd as xls
import numpy as np

In [190]:
file = "/content/existencias-animales.xlsx"

In [191]:
get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot]
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

Obtaining indexes for sectoring data content

In [192]:
work_book = xls.open_workbook(file)
work_book = work_book.sheet_by_index(0)

column = 1

array_col = work_book.col_values(column)[1:]
ini = get_ini("", array_col)[-1]
fin = get_indexes("", array_col)[-1]

array_col = array_col[ini: fin]

print(array_col)

titles = array_col

['Total Nacional - Región', 'Total Nacional', 'Región de Arica y Parinacota', 'Región de Tarapacá ', 'Región de Antofagasta', 'Región de Atacama', 'Región de Coquimbo', 'Región de Valparaíso', 'Región Metropolitana de Santiago', "Región del Libertador General Bernardo O'Higgins", 'Región del Maule', 'Región de Ñuble', 'Región del Biobío ', 'Región de La Araucanía', 'Región de Los Ríos', 'Región de Los Lagos', 'Región de Aysén del General Carlos Ibáñez del Campo', 'Región de Magallanes y de la Antártica Chilena']


Selecting columns as arrays
Sizing data of the columns
Data normalization
Storing arrays into fields

This process will finish when there are no more columns with data

In [193]:
fields = np.array(array_col)

column=2
while True:
  try:
    array_col = work_book.col_values(column)[1:]
    array_col = array_col[ini:fin]
    column+=1
    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.replace(array_col, " ", "_")
    #print(array_col)
    fields = np.row_stack((fields, array_col))
  except:
    print("Ending column")
    break

Ending column


In [194]:
array_to_dataframe = np.transpose(fields)
titles = array_to_dataframe[0]
print(titles)


df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
df_polar = df_polar[1:]
print(df_polar)

df_polar.write_csv(file.split('.')[0]+'.csv')


['Total Nacional - Región' 'bovinos' 'ovinos' 'caprinos' 'porcinos'
 'caballares' 'mulares_y_burdéganos' 'asnales' 'llamas' 'alpacas'
 'huarizos' 'conejos' 'jabalíes' 'ciervos' 'gallinas/pollos_4_' 'pavos'
 'otras_aves_de_corral_5']
shape: (17, 17)
┌────────────┬───────────┬───────────┬──────────┬─────┬─────────┬────────────┬───────────┬────────────┐
│ Total      ┆ bovinos   ┆ ovinos    ┆ caprinos ┆ ... ┆ ciervos ┆ gallinas/p ┆ pavos     ┆ otras_aves │
│ Nacional - ┆ ---       ┆ ---       ┆ ---      ┆     ┆ ---     ┆ ollos_4_   ┆ ---       ┆ _de_corral │
│ Región     ┆ str       ┆ str       ┆ str      ┆     ┆ str     ┆ ---        ┆ str       ┆ _5         │
│ ---        ┆           ┆           ┆          ┆     ┆         ┆ str        ┆           ┆ ---        │
│ str        ┆           ┆           ┆          ┆     ┆         ┆            ┆           ┆ str        │
╞════════════╪═══════════╪═══════════╪══════════╪═════╪═════════╪════════════╪═══════════╪════════════╡
│ Total      ┆ 2474406.